<a href="https://colab.research.google.com/github/JuanPabl07DP/SimulacionME/blob/main/Simulaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulación de bot para el monitoreo de emociones (Prototipo)

## Desarrollado por:
- Juan Pablo Daza Pereira
- Juan Sebastian Camargo Sanchez

## Subir archivos

In [36]:
from google.colab import files

#print("🔼 Sube una imagen (con rostro):")
#uploaded_img = files.upload()

# Obtener el nombre del archivo
#image_file = list(uploaded_img.keys())[0]

# Leer imagen como bytes
#with open(image_file, 'rb') as image:
#    image_bytes = image.read()

#Para audio
print("🔼 Sube un archivo de audio (.wav o .mp3):")
uploaded_audio = files.upload()

audio_file = list(uploaded_audio.keys())[0]

🔼 Sube un archivo de audio (.wav o .mp3):


Saving record_out (1).wav to record_out (1).wav


## API Keys

In [ ]:
GOOGLE_VISION_API_KEY = ""
ASSEMBLYAI_API_KEY = ""
AZURE_FACE_API_KEY = ""
AZURE_FACE_API_ENDPOINT = ""
AWS_ACCESS_KEY_ID = ""
AWS_SECRET_ACCESS_KEY = ""
AWS_REGION = ""


## Envío de imagenes a Google Cloud Vision API

In [ ]:
import requests
import base64
import json

# Obtener nombre del archivo
image_file = list(uploaded_img.keys())[0]

# Codificar imagen a base64
with open(image_file, "rb") as image:
    img_base64 = base64.b64encode(image.read()).decode()

# Preparar solicitud
vision_url = f"https://vision.googleapis.com/v1/images:annotate?key={GOOGLE_VISION_API_KEY}"

payload = {
    "requests": [{
        "image": {
            "content": img_base64
        },
        "features": [{
            "type": "FACE_DETECTION",
            "maxResults": 1
        }]
    }]
}

response = requests.post(vision_url, json=payload)
face_data = response.json()

# Verifica si la respuesta contiene 'responses'
if "responses" in face_data:
    response_data = face_data["responses"][0]

    if "faceAnnotations" in response_data:
        emotions = response_data["faceAnnotations"][0]
        print("Emociones detectadas en la imagen:")
        for emotion in ['joyLikelihood', 'sorrowLikelihood', 'angerLikelihood', 'surpriseLikelihood']:
            print(f"{emotion}: {emotions[emotion]}")
    else:
        print("No se detectó ningún rostro en la imagen.")
else:
    print("Error en la respuesta de la API:")
    print(json.dumps(face_data, indent=2))


Emociones detectadas en la imagen:
joyLikelihood: VERY_LIKELY
sorrowLikelihood: VERY_UNLIKELY
angerLikelihood: VERY_UNLIKELY
surpriseLikelihood: VERY_UNLIKELY


## Envío de imagenes a Face API

In [ ]:
import requests
import json
# Ruta de la imagen subida en Colab
image_path = list(uploaded_img.keys())[0]

# Leer imagen en binario
with open(image_path, "rb") as image_file:
    image_data = image_file.read()

# Configurar headers y parámetros de Face API
headers = {
    "Ocp-Apim-Subscription-Key": AZURE_FACE_API_KEY,
    "Content-Type": "application/octet-stream"
}
params = {
    "returnFaceAttributes": "emotion"
}

# Hacer solicitud POST
face_api_url = f"{AZURE_FACE_API_ENDPOINT}/face/v1.0/detect"
response = requests.post(face_api_url, params=params, headers=headers, data=image_data)

# Procesar respuesta
if response.status_code == 200:
    results = response.json()
    if results:
        print("Emociones detectadas con Azure Face API:")
        emotions = results[0]["faceAttributes"]["emotion"]
        for emotion, value in emotions.items():
            print(f"{emotion.capitalize()}: {value:.3f}")
    else:
        print("No se detectaron rostros en la imagen.")
else:
    print("Error al consultar Azure Face API:")
    print(response.text)

Error al consultar Azure Face API:
{
  "error": {
    "code": "InvalidRequest",
    "message": "Invalid request has been sent.",
    "innererror": {
      "code": "UnsupportedFeature",
      "message": "Return Attributes (emotion, gender, age, smile, facial hair, hair and makeup) have been deprecated and are no longer supported. For more information, please see https://aka.ms/facerecognition."
    }
  }
}


## Envío de imagenes a Amazon Rekognition

In [ ]:
!pip install boto3

In [ ]:
import boto3
# Crear cliente de Rekognition
rekognition = boto3.client(
    'rekognition',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION
)

# Enviar solicitud para detectar emociones
response = rekognition.detect_faces(
    Image={'Bytes': image_bytes},
    Attributes=['ALL']
)

# Mostrar emociones
if 'FaceDetails' in response and len(response['FaceDetails']) > 0:
    print("Emociones detectadas con Amazon Rekognition:")
    emotions = response['FaceDetails'][0]['Emotions']
    for emotion in sorted(emotions, key=lambda x: x['Confidence'], reverse=True):
        print(f"{emotion['Type']}: {emotion['Confidence']:.2f}%")
else:
    print("⚠️ No se detectaron rostros en la imagen.")

Emociones detectadas con Amazon Rekognition:
HAPPY: 96.42%
CONFUSED: 0.44%
CALM: 0.28%
SURPRISED: 0.15%
SAD: 0.12%
DISGUSTED: 0.03%
ANGRY: 0.00%
FEAR: 0.00%


## Envío de audio a AssemblyAI

In [ ]:
import time
import requests

audio_file = list(uploaded_audio.keys())[0]

# Subir archivo a AssemblyAI
headers_auth = {'authorization': ASSEMBLYAI_API_KEY}
upload_url = "https://api.assemblyai.com/v2/upload"

with open(audio_file, 'rb') as f:
    upload_response = requests.post(upload_url, headers=headers_auth, files={'file': f})

audio_url = upload_response.json()['upload_url']

# Solicitar transcripción con análisis de emociones
transcribe_url = "https://api.assemblyai.com/v2/transcript"
payload = {
    "audio_url": audio_url,
    "speaker_labels": False,

    "iab_categories": True,
    "auto_highlights": True,
    "sentiment_analysis": True
}

transcribe_response = requests.post(transcribe_url, json=payload, headers=headers_auth)
transcript_id = transcribe_response.json()['id']

# Esperar a que termine
print("Analizando audio...")
status = "processing"
while status != "completed":
    polling = requests.get(f"https://api.assemblyai.com/v2/transcript/{transcript_id}", headers=headers_auth)
    result = polling.json()
    status = result["status"]
    time.sleep(3)

# Mostrar emociones
print("Emociones detectadas en el audio:")
for sentiment in result.get("sentiment_analysis_results", []):
    print(f"Texto: {sentiment['text']}")
    print(f"Emoción: {sentiment['sentiment']} | Confianza: {sentiment['confidence']:.2f}\n")

Analizando audio...
Emociones detectadas en el audio:
Texto: Ola minombres juan paolo yesoy muifari.
Emoción: NEUTRAL | Confianza: 0.78



## Envío de audio a Vokaturi

In [31]:
import os
import sys
import zipfile
import subprocess
from google.colab import files
import wave
import numpy as np
import ctypes

In [32]:
zip_path = '/content/OpenVokaturi-4-0.zip'
if not os.path.exists(zip_path):
    print("Descargando Vokaturi...")
    !wget -O /content/OpenVokaturi-4-0.zip https://developers.vokaturi.com/downloads/OpenVokaturi-4-0.zip

# Descomprimir
if os.path.exists(zip_path):
    print(f"Descomprimiendo {zip_path}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('/content')
    print("Vokaturi descomprimido correctamente")
else:
    print("Error: No se pudo descargar Vokaturi")

Descomprimiendo /content/OpenVokaturi-4-0.zip...
Vokaturi descomprimido correctamente


In [33]:
VOKATURI_API_PATH = "/content/OpenVokaturi-4-0/api"
VOKATURI_LIB_PATH = "/content/OpenVokaturi-4-0/lib/open/linux/OpenVokaturi-4-0-linux.so"

sys.path.append(VOKATURI_API_PATH)

# Verificar que existe y luego importar
if os.path.exists(VOKATURI_API_PATH + "/Vokaturi.py"):
    import Vokaturi
    print("✓ Vokaturi importado exitosamente")
else:
    print("✗ Error: No se encontró Vokaturi.py")
    print(f"Buscando en: {VOKATURI_API_PATH}")

✓ Vokaturi importado exitosamente


In [37]:
def convertir_a_wav(archivo_entrada, archivo_salida):
    try:
        # Verificar si ffmpeg está instalado
        try:
            subprocess.run(['ffmpeg', '-version'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        except FileNotFoundError:
            print("Instalando ffmpeg...")
            subprocess.run(['apt-get', 'update', '-qq'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            subprocess.run(['apt-get', 'install', '-y', 'ffmpeg'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Convertir a WAV mono de 16-bit
        comando = [
            'ffmpeg',
            '-i', archivo_entrada,
            '-ac', '1',                # Mono
            '-ar', '44100',            # 44.1kHz
            '-acodec', 'pcm_s16le',    # 16-bit PCM
            '-y',                      # Sobrescribir
            archivo_salida
        ]

        resultado = subprocess.run(comando,
                                  stdout=subprocess.PIPE,
                                  stderr=subprocess.PIPE,
                                  text=True)

        if resultado.returncode != 0:
            print(f"Error al convertir: {resultado.stderr}")
            return False

        print(f"Archivo convertido a WAV")
        return True

    except Exception as e:
        print(f"Error en conversión: {str(e)}")
        return False

def analizar_emociones_audio(ruta_archivo):
    # Verificar el archivo
    if not os.path.exists(ruta_archivo):
        return {"error": f"Archivo no encontrado: {ruta_archivo}"}

    # Cargar la biblioteca (Vokaturi ya está importado arriba)
    try:
        Vokaturi.load(VOKATURI_LIB_PATH)
        print("Biblioteca Vokaturi cargada")
    except Exception as e:
        return {"error": f"Error cargando Vokaturi: {str(e)}"}

    try:
        # Leer el archivo WAV
        archivo = wave.open(ruta_archivo, "rb")

        # Obtener propiedades
        frecuencia = archivo.getframerate()
        muestras_total = archivo.getnframes()
        canales = archivo.getnchannels()
        duracion = muestras_total / frecuencia

        print(f"Audio: {duracion:.2f}s, {frecuencia}Hz, {canales} canal(es)")

        # Leer datos
        buffer = archivo.readframes(muestras_total)
        archivo.close()

        # Convertir a valores numéricos
        muestras = np.frombuffer(buffer, dtype=np.int16)

        # Convertir a mono si es estéreo
        if canales == 2:
            print("Convirtiendo estéreo a mono...")
            muestras = np.mean(muestras.reshape(-1, 2), axis=1).astype(np.int16)

        # Crear objeto Voice
        voz = Vokaturi.Voice(frecuencia, len(muestras), 1)

        # Convertir a formato C
        muestras_c = (ctypes.c_short * len(muestras))()
        for i in range(len(muestras)):
            muestras_c[i] = muestras[i]

        # Analizar
        voz.fill_int16array(len(muestras), muestras_c)

        # Objetos para resultados
        calidad = Vokaturi.Quality()
        emociones_prob = Vokaturi.EmotionProbabilities()

        # Extraer emociones
        voz.extract(calidad, emociones_prob)

        # Guardar resultados
        emociones = {
            "Neutral": emociones_prob.neutrality,
            "Felicidad": emociones_prob.happiness,
            "Tristeza": emociones_prob.sadness,
            "Ira": emociones_prob.anger,
            "Miedo": emociones_prob.fear
        }

        # Liberar recursos
        voz.destroy()

        return {"emociones": emociones}

    except Exception as e:
        print(f"Error en análisis: {str(e)}")
        import traceback
        traceback.print_exc()
        return {"error": str(e)}

In [38]:
if audio_file:  # Solo proceder si se subió un archivo
    print("\nAnalizando el archivo de audio...")

    # Verificar si es WAV o MP3
    extension = os.path.splitext(audio_file)[1].lower()
    archivo_wav = audio_file

    # Convertir si es MP3
    if extension == '.mp3':
        print("Detectado archivo MP3, convirtiendo...")
        archivo_wav = audio_file.replace('.mp3', '.wav')
        if not convertir_a_wav(audio_file, archivo_wav):
            print("No se pudo convertir el archivo")
            archivo_wav = None
    elif extension != '.wav':
        print(f"Formato no soportado: {extension}")
        print("Solo se admiten archivos .wav o .mp3")
        archivo_wav = None

    # Analizar si tenemos un archivo WAV
    if archivo_wav:
        resultado = analizar_emociones_audio(archivo_wav)

        if isinstance(resultado, dict) and 'error' in resultado:
            print(f"Error: {resultado['error']}")
        else:
            # Ordenar solo las emociones
            emociones = {
                "NEUTRAL": resultado['emociones']['Neutral'] * 100,
                "HAPPY": resultado['emociones']['Felicidad'] * 100,
                "SAD": resultado['emociones']['Tristeza'] * 100,
                "ANGRY": resultado['emociones']['Ira'] * 100,
                "FEAR": resultado['emociones']['Miedo'] * 100
            }

            # Ordenar por valor
            emociones_ordenadas = sorted(emociones.items(), key=lambda x: x[1], reverse=True)

            print("\nEmociones detectadas con Vokaturi:")
            for emocion, valor in emociones_ordenadas:
                print(f"{emocion}: {valor:.2f}%")
else:
    print("No se subió ningún archivo de audio")


Analizando el archivo de audio...
Biblioteca Vokaturi cargada
Audio: 7.68s, 48000Hz, 1 canal(es)

Emociones detectadas con Vokaturi:
ANGRY: 80.85%
HAPPY: 19.13%
SAD: 0.01%
NEUTRAL: 0.00%
FEAR: 0.00%
